# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 12:08PM,242753,10,MSP212545,"Methapharm, Inc.",Released
1,Jul 26 2022 12:08PM,242753,10,MSP212535,"Methapharm, Inc.",Released
2,Jul 26 2022 12:08PM,242753,10,MSP212537,"Methapharm, Inc.",Released
3,Jul 26 2022 11:57AM,242752,10,MSP212538,"Methapharm, Inc.",Released
4,Jul 26 2022 11:57AM,242752,10,MSP212539,"Methapharm, Inc.",Released
5,Jul 26 2022 11:57AM,242752,10,MSP212543,"Methapharm, Inc.",Released
6,Jul 26 2022 11:57AM,242752,10,MSP212544,"Methapharm, Inc.",Released
7,Jul 26 2022 11:57AM,242752,10,MSP212540,"Methapharm, Inc.",Released
8,Jul 26 2022 11:57AM,242752,10,MSP212541,"Methapharm, Inc.",Released
9,Jul 26 2022 11:57AM,242752,10,MSP212536,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,242747,Executing,2
35,242747,Released,17
36,242751,Released,2
37,242752,Released,19
38,242753,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242746,NaN,NaN,1.0
242747,NaN,2.0,17.0
242751,NaN,NaN,2.0
242752,NaN,NaN,19.0
242753,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242685,0.0,1.0,0.0
242696,45.0,8.0,0.0
242711,0.0,24.0,1.0
242717,0.0,10.0,2.0
242718,0.0,27.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242685,0,1,0
242696,45,8,0
242711,0,24,1
242717,0,10,2
242718,0,27,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242685,0,1,0
1,242696,45,8,0
2,242711,0,24,1
3,242717,0,10,2
4,242718,0,27,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242685,,1,
1,242696,45,8,
2,242711,,24,1
3,242717,,10,2
4,242718,,27,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc."
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc."
22,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc."
24,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd"
43,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc."
44,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC"
45,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation
51,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC"
52,Jul 26 2022 11:01AM,242741,10,Bio-PRF
63,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",,,3
1,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",,,19
2,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",,,2
3,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",,2,17
4,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",,,1
5,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC",,,1
6,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,,6,
7,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",,,1
8,Jul 26 2022 11:01AM,242741,10,Bio-PRF,,,11
9,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3,,
1,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19,,
2,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2,,
3,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",17,2,
4,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",1,,
5,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC",1,,
6,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,,6,
7,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1,,
8,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11,,
9,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3,,
1,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19,,
2,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2,,
3,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",17,2,
4,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3.0,NaN,NaN
1,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19.0,NaN,NaN
2,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2.0,NaN,NaN
3,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",17.0,2.0,NaN
4,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3.0,0.0,0.0
1,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19.0,0.0,0.0
2,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2.0,0.0,0.0
3,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",17.0,2.0,0.0
4,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3155569,51.0,23.0,45.0
12,242740,0.0,1.0,0.0
15,1213652,38.0,34.0,0.0
16,485442,4.0,24.0,0.0
18,242685,0.0,1.0,0.0
19,485486,18.0,2.0,0.0
20,1213589,96.0,61.0,0.0
21,242746,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3155569,51.0,23.0,45.0
1,12,242740,0.0,1.0,0.0
2,15,1213652,38.0,34.0,0.0
3,16,485442,4.0,24.0,0.0
4,18,242685,0.0,1.0,0.0
5,19,485486,18.0,2.0,0.0
6,20,1213589,96.0,61.0,0.0
7,21,242746,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,51.0,23.0,45.0
1,12,0.0,1.0,0.0
2,15,38.0,34.0,0.0
3,16,4.0,24.0,0.0
4,18,0.0,1.0,0.0
5,19,18.0,2.0,0.0
6,20,96.0,61.0,0.0
7,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,12,Released,0.0
2,15,Released,38.0
3,16,Released,4.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,23.0,1.0,34.0,24.0,1.0,2.0,61.0,0.0
Released,51.0,0.0,38.0,4.0,0.0,18.0,96.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,1.0,34.0,24.0,1.0,2.0,61.0,0.0
2,Released,51.0,0.0,38.0,4.0,0.0,18.0,96.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,1.0,34.0,24.0,1.0,2.0,61.0,0.0
2,Released,51.0,0.0,38.0,4.0,0.0,18.0,96.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()